In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import sweetviz as sv

import scarab.data as scd
import scarab.matching as scm

In [3]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

sc_discrete_colors = [
        '#0CD3FA',
        '#1194AD',
        '#012BFE',
        '#B000F9',
        '#3411AD',
        '#F900A0', # highlight pink
        '#00F9B0', # highlight turquoise
        '#F9B000'  # casino gold
    ]

pio.templates['sc_colors'] = go.layout.Template(
    layout_paper_bgcolor='#E5ECF6', # grey
    layout_plot_bgcolor='#E5ECF6',  # grey
    layout_colorway=sc_discrete_colors,
    layout_colorscale={'sequential':'teal'})
pio.templates.default = 'plotly+sc_colors'

# Player data

In [312]:
filename = scd.data_repository_path()
filename += '\\raw\\ims\\ogn\\'
filename += 'OGN_report_Reporting_-_Players_2022-09-12_0805.csv'

cols_date = ['Birth date','Signup date','Last login date','Last game date']
cols_rename = {'E-mail': 'email',
            'E-mail verified': 'email_verified',
            'Cellphone': 'mobile'}
df_ogn = pd.read_csv(filename, parse_dates=cols_date)
df_ogn.shape


(11208, 22)

In [313]:
df_ogn['Last login date'] = pd.to_datetime(df_ogn['Last login date'].replace(' ', '')) 
df_ogn['Last game date'] = pd.to_datetime(df_ogn['Last login date'].replace(' ', '')) 
df_ogn = df_ogn.rename(columns=cols_rename).rename(columns=scd.lower_case_underscore)

mask_playtech = df_ogn.email.str.contains('playtech', case=False)
mask_not_verified = df_ogn.email_verified == 'No'
mask_before_2022 = df_ogn.signup_date.dt.year < 2022
mask_remove = mask_playtech | mask_not_verified | mask_before_2022

print('Playtech accounts:', sum(mask_playtech))
print('Email not verified:', sum(mask_not_verified))
print('Signup before 2022:', sum(mask_before_2022))
print('Total accounts removed:', sum(mask_remove))

df_ogn = df_ogn[~mask_remove]
scd.print_dataframe_overview(df_ogn)

Playtech accounts: 32
Email not verified: 1635
Signup before 2022: 377
Total accounts removed: 1796
(9412, 22)
NaN entries:
last_login_date    402
last_game_date     402
dtype: int64


,playercode,username,email,email_verified,birth_date,age,country,country_code,first_name,last_name,...,signup_date,last_login_date,last_game_date,gamecount,bets,wins,deposit_count,total_deposits,total_withdraws,net_loss
337,11311543,Anastasia,HumorvollSchoenerBasilisk@magspam.net,Yes,1960-01-01,62,Switzerland,CH,Sara,Muster,...,2022-01-03 19:06:29,NaT,NaT,0,0.0,0.0,23,22122,22122.0,0.0
344,11311675,HugoBoss,AnziehendStarkerFlamingo@10minmail.de,Yes,1960-01-01,62,Switzerland,CH,Elle,Muster,...,2022-01-03 19:44:28,NaT,NaT,0,0.0,0.0,23,22122,22122.0,0.0
347,11320734,Mona1,simona.i@gmx.ch,Yes,1985-01-26,37,Switzerland,CH,Simona,Iorlano,...,2022-01-07 17:54:39,2022-01-07 17:56:21,2022-01-07 17:56:21,0,0.0,0.0,23,22122,23122.0,-1000.0
349,11322642,Saletti1234,rcivkin@rocketmail.com,Yes,1998-03-21,24,Switzerland,CH,Robin,Civkin,...,2022-01-08 11:23:30,2022-06-19 18:18:45,2022-06-19 18:18:45,33,76175.0,69650.0,27,26122,25597.0,525.0
350,11313174,Meliii,melinahauri15@gmail.com,Yes,2001-07-21,21,Switzerland,CH,Melina,Hauri,...,2022-01-04 13:50:44,2022-07-17 17:56:39,2022-07-17 17:56:39,46,89578.0,83779.0,30,29122,30073.0,-3201.0
351,11328923,Salvo,toto2009@gmx.ch,Yes,1979-02-20,43,Switzerland,CH,Salvatore,Corso,...,2022-01-10 20:52:48,2022-07-17 18:13:26,2022-07-17 18:13:26,30,19620.0,35215.0,28,27122,46717.0,-20595.0
353,11323707,purzel,web23@gmx.ch,Yes,1965-09-29,57,Switzerland,CH,Gabriela,Gabi,...,2022-01-08 19:06:28,2022-02-20 19:45:58,2022-02-20 19:45:58,8,4096.0,4879.0,23,22122,24905.0,-2783.0
354,11318964,Trevi72,andrea.trevisan1972@gmail.com,Yes,1972-12-05,50,Switzerland,CH,Andreas,Trevisan,...,2022-01-06 20:56:04,2022-02-20 19:05:16,2022-02-20 19:05:16,8,6118.0,21208.0,23,22122,39212.0,-17090.0
355,11319013,Sotti,sonia.trevisan76@gmail.com,Yes,1976-06-26,46,Switzerland,CH,Sonia,Trevisan,...,2022-01-06 21:09:21,2022-02-20 19:09:44,2022-02-20 19:09:44,6,4102.0,3102.0,23,22122,23122.0,-1000.0
356,11325402,habeeby,bbryadh2@hotmail.com,Yes,1986-05-26,36,Switzerland,CH,Ryadh,Ben Brahim,...,2022-01-09 12:45:41,2022-02-20 19:57:08,2022-02-20 19:57:08,3,3000.0,4290.0,23,22122,25412.0,-3290.0


In [314]:
df_ogn[df_ogn.username.str.contains('test', case=False) | df_ogn.email.str.contains('test', case=False)].head()

,playercode,username,email,email_verified,birth_date,age,country,country_code,first_name,last_name,...,signup_date,last_login_date,last_game_date,gamecount,bets,wins,deposit_count,total_deposits,total_withdraws,net_loss
476,11317680,Test0101,max.stuerzekarn@gmail.com,Yes,1989-07-27,33,Switzerland,CH,Maximilian,Stürzekarn,...,2022-01-06 10:49:16,2022-01-06 11:23:39,2022-01-06 11:23:39,0,0.0,0.0,23,22122,23122.0,-1000.0
754,11356216,sctest102,tarkan.oezkan@scgroup.ch,Yes,1994-12-12,28,Switzerland,CH,Tarkan,Özkan,...,2022-01-13 20:40:19,2022-07-21 15:17:02,2022-07-21 15:17:02,0,0.0,0.0,23,22122,26122.0,-4000.0
1241,11376595,Chica,test@10minutenmail.xyz,Yes,1989-08-10,33,Switzerland,CH,Test,Blatter,...,2022-01-19 14:02:39,2022-01-19 14:10:11,2022-01-19 14:10:11,0,0.0,0.0,23,22122,23122.0,-1000.0
1848,11705676,user911,vicky.test001@yahoo.com,Yes,1996-08-11,26,Switzerland,CH,Lol,Lol,...,2022-02-15 14:15:27,2022-08-16 09:31:26,2022-08-16 09:31:26,0,0.0,0.0,18,17123,19123.0,-2000.0
2429,11782891,gfromro,testingfromro@mailinator.com,Yes,1993-03-23,29,Switzerland,CH,Catalin,Hercut,...,2022-02-18 08:58:18,2022-02-18 09:24:35,2022-02-18 09:24:35,0,0.0,0.0,14,13123,14123.0,-1000.0


In [214]:
report_player = sv.analyze(df_ogn)
report_player.show_html(filepath=r'sweetviz_players_data.html', 
            open_browser=True, 
            layout='vertical')

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)


Report sweetviz_players_data.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [215]:
cols_hist = ['signup_date', 'last_login_date', 'last_game_date']
for c in cols_hist:
    px.histogram(df_ogn, x=c).show()

# Player/game data

In [352]:
filename = scd.data_repository_path()
filename += '\\raw\\ims\\ogn\\'
filename += 'OGN_report_Reporting_-_Game_statistics_2022-09-13_0624.csv'

cols_date = ['Date']
cols_rename = {'Games(#)': 'games'}
df_game = pd.read_csv(filename, parse_dates=cols_date)

In [353]:
df_game = df_game.rename(columns=cols_rename).rename(columns=scd.lower_case_underscore).drop(columns=['currency'])

mask_before_2022 = df_game.date.dt.year < 2022
mask_not_in_player_data = ~df_game.username.isin(df_ogn.username.unique())
mask_remove = mask_before_2022 | mask_not_in_player_data

print('Before 2022:', sum(mask_before_2022))
print('Not in player data:', sum(mask_not_in_player_data))
print('Entries removed:', sum(mask_remove))

df_game = df_game[~mask_remove]
scd.print_dataframe_overview(df_game)

Before 2022: 100
Not in player data: 475
Entries removed: 475
(9746, 7)


,username,player_code,date,games,total_bets,total_wins,total_income
101,Simba8,11315298,2022-01-05,13,6340.0,4855.0,1485.0
109,Dosimusmaximus,11315300,2022-01-05,6,2460.0,2100.0,360.0
112,Lucky,11349991,2022-01-13,9,6965.0,8635.0,-1670.0
113,SG007,11355389,2022-01-13,3,1210.0,210.0,1000.0
114,Stoki,11355330,2022-01-13,2,1000.0,0.0,1000.0
116,Sandor,11355343,2022-01-13,2,2210.0,620.0,1590.0
119,zuespa,11354918,2022-01-13,9,3840.0,2325.0,1515.0
120,Coach22,11344403,2022-01-13,7,1455.0,4890.0,-3435.0
121,Guschti,11315430,2022-01-13,9,7275.0,12750.0,-5475.0
122,JokerNa,11348903,2022-01-13,9,8170.0,5680.0,2490.0


In [11]:
report_game = sv.analyze(df_game)
report_game.show_html(filepath=r'sweetviz_games_data.html', 
            open_browser=True, 
            layout='vertical')

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:01 -> (00:00 left)


Report sweetviz_games_data.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [12]:
cols_hist = ['date']
for c in cols_hist:
    px.histogram(df_game, x=c).show()

# Online Casino data

In [103]:
filename = scd.data_repository_path()
filename += '\\processed\\der\\'
filename += 'oc_der_regulation_status_20220531.csv'
df_oc = pd.read_csv(filename, parse_dates=['birthdate','signupdate','regulation_status_date','player_personal_sync_date'])

C:\Users\LEON~1.PEE\AppData\Local\Temp/ipykernel_30628/931838732.py:4: DtypeWarning:

Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.



In [120]:
df_oc = df_oc[df_oc.regulation_status.isin(['closed_verified_account','provisional_account','verified_account'])]
scd.print_dataframe_overview(df_oc)
df_oc.regulation_status.value_counts()

(84866, 18)
NaN entries:
citizenship    8
zip            9
dtype: int64


,code,username,first_name,last_name,birthdate,citizenship,city,countryname,gender,zip,signupdate,regulation_status,regulationstatuscode,player_personal_sync_date,regulation_status_date,birth_date,cleanfullname,fingerprint
0,13426792,StiffM0tho,Mario,Stalder,1997-05-06,CH,Schaffhausen,Switzerland,M,8200,2019-09-02 12:41:40,verified_account,CHE-2,2022-08-15 00:16:33,2019-10-08,1997-05-06,mario stalder,mariostalder19970506M
1,13426796,ale979,alessandra,farina,1979-09-25,CH,Lugano,Switzerland,F,6900,2019-09-02 12:42:29,closed_verified_account,CHE-3,2022-06-25 01:08:55,2019-11-06,1979-09-25,alessandra farina,alessandrafarina19790925F
2,13426905,yoko1,Katiucia,Girardin,1989-09-14,CH,Alpnachstad,Switzerland,F,6053,2019-09-02 13:18:27,closed_verified_account,CHE-3,2022-04-19 09:05:41,2019-10-20,1989-09-14,girardin katiucia,girardinkatiucia19890914F
3,13426944,paolopirlo,Frederic,Rochat,1983-10-03,CH,Le Mont-sur-Lausanne,Switzerland,M,1052,2019-09-02 13:31:49,verified_account,CHE-2,2022-09-13 21:36:47,2020-12-01,1983-10-03,frederic rochat,fredericrochat19831003M
4,13427401,Activ,Alex,Hoang,1993-04-06,VN,Eschenbach SG,Switzerland,M,8733,2019-09-02 15:13:36,closed_verified_account,CHE-3,2022-04-19 07:38:19,2020-07-03,1993-04-06,alex hoang,alexhoang19930406M
5,13427415,pgre9150,Patrick,Greuter,1984-11-14,CH,Winterthur,Switzerland,M,8408,2019-09-02 15:17:26,verified_account,CHE-2,2022-09-13 16:31:38,2019-10-08,1984-11-14,greuter patrick,greuterpatrick19841114M
6,13427658,Patrick8282,Patrick,Widmer,1995-11-28,CH,Wädenswil,Switzerland,M,8820,2019-09-02 15:50:43,verified_account,CHE-2,2022-09-13 15:03:06,2019-10-08,1995-11-28,patrick widmer,patrickwidmer19951128M
7,13427873,Moekli,Martin,Mock,1981-02-20,CH,Tuggen,Switzerland,M,8856,2019-09-02 16:14:44,verified_account,CHE-2,2022-09-12 01:16:57,2019-10-08,1981-02-20,martin mock,martinmock19810220M
8,13435659,pt_test_int_22,dsgsfdh,dfhgj,1992-07-06,CH,Lenzburg,Switzerland,M,5600,2019-09-04 06:53:23,verified_account,CHE-2,2022-07-25 00:14:11,2019-09-04,1992-07-06,dfhgj dsgsfdh,dfhgjdsgsfdh19920706M
9,13435708,Arthur2019,Martin,Dobler,1981-12-31,AT,Diepoldsau,Switzerland,M,9444,2019-09-04 07:18:26,verified_account,CHE-2,2022-06-17 05:00:16,2019-10-08,1981-12-31,dobler martin,doblermartin19811231M


verified_account           50906
closed_verified_account    30092
provisional_account         3868
Name: regulation_status, dtype: int64

# Duplicates and matches

## Duplicates in the OGN player data

In [315]:
df_ogn = df_ogn.rename(columns={'firstname': 'first_name',
                        'lastname': 'last_name'})
df_ogn['birth_date'] = df_ogn.birth_date.astype(str)
# preprocess the dataframe to create the identity "fingerprint"
df_ogn = scm.preprocess_identity_dataframe(df_ogn)
# vectorize the fingerprints using td/idf
m_tf_idf = scm.tf_idf_vectorize(df_ogn.fingerprint)

In [316]:
# compute the best score (cosine similarity) matches between the dataframes  
df_best_match = scm.compute_best_cosine_matches(m_tf_idf, m_tf_idf, num_matches=4)
# remove lower diagonal and the diagonal itself
df_best_match = df_best_match[df_best_match.idx < df_best_match.match_idx].reset_index(drop=True)

Duration:  0.92 s


In [317]:
# join the dataframes on the best match
df_match = scm.join_matches(df_best_match,
                            df_ogn,
                            df_ogn)
# extend the data with two additional scores for (1) the date-of-birth similarity and (2) the full-name similarity 
df_match['dob_distance'] = df_match.apply(lambda x: scm.dob_distance(x.birth_date_m, x.birth_date_o), axis=1)
# max_lev_weight_bipartite_matching is computationally expensive, so only compute it where needed (dob_distance=0)
df_match['me_score'] = 0
df_match.loc[df_match.dob_distance==0,'me_score'] = df_match[df_match.dob_distance==0].apply(
                                                        lambda x: scm.max_lev_weight_bipartite_matching(
                                                            x.cleanfullname_m,
                                                            x.cleanfullname_o),
                                                        axis=1)

In [318]:
df_match['score_bin'] = pd.cut(df_match.score, np.linspace(0.5,1.05,12))
df_match['me_score_bin'] = pd.cut(df_match.me_score, np.linspace(0.5,1.05,12))
pd.pivot_table(df_match[df_match.dob_distance==0],
                index='score_bin',
                columns='me_score_bin',
                values='score',
                aggfunc='count')

me_score_bin,"(0.5, 0.55]","(0.55, 0.6]","(0.6, 0.65]","(0.65, 0.7]","(0.7, 0.75]","(0.75, 0.8]","(0.8, 0.85]","(0.85, 0.9]","(0.9, 0.95]","(0.95, 1.0]","(1.0, 1.05]"
score_bin,,,,,,,,,,,
"(0.5, 0.55]",1,3,2,0,0,0,0,0,0,0,0
"(0.55, 0.6]",1,7,3,1,0,1,0,1,0,1,0
"(0.6, 0.65]",0,10,4,2,2,0,0,0,0,2,0
"(0.65, 0.7]",0,2,3,1,1,0,0,1,0,2,0
"(0.7, 0.75]",0,3,1,1,1,0,1,0,0,1,0
"(0.75, 0.8]",0,1,0,3,1,0,2,2,2,8,0
"(0.8, 0.85]",0,1,0,0,0,0,1,2,0,5,0
"(0.85, 0.9]",0,0,0,0,0,0,0,2,6,4,0
"(0.9, 0.95]",0,0,1,0,0,0,1,0,1,1,0


In [319]:
# apply the same cutoffs as for BSI/IMS matching, which
# (1) have been validated extensively, and
# (2) seem to make sense for this case as well based on the pivot table above 
mask_match = ((df_match.dob_distance==0) 
            & (df_match.score.between(0.0,1.1)
            & df_match.me_score.between(0.9,1.1)))

master_sort_config = {'column': 'signup_date', 'ascending': False}
df_ogn = scm.process_duplicate_identities(df_ogn, df_match[mask_match], master_sort_config)

Number of components: 332


In [320]:
df_ogn_user_to_master = pd.merge(df_ogn.filter(['username','idx_master']),
                                df_ogn.filter(['username','idx']),
                                left_on='idx_master', right_on='idx',
                                how='inner',
                                suffixes=['','_master']).drop(columns=['idx_master','idx'])
df_ogn_user_to_master.head()

,username,username_master
0,Anastasia,Anastasia
1,HugoBoss,HugoBoss
2,Mona1,Mona1
3,Saletti1234,Saletti1234
4,Meliii,Meliii


In [321]:
print('Number of players with duplicates:',df_ogn[df_ogn.component>=0].shape[0])
print('Number of master records:',df_ogn[df_ogn.component>=0].is_master.sum())

Number of players with duplicates: 695
Number of master records: 332


In [322]:
def create_master_player_records(df_ogn):
    master_agg_dict = {
        'signup_date': 'min',
        'last_login_date': 'max',
        'last_game_date': 'max',
        'gamecount': 'sum',
        'bets': 'sum',
        'wins': 'sum',
        'deposit_count': 'sum',
        'total_deposits': 'sum',
        'total_withdraws': 'sum',
        'net_loss': 'sum'}
    df_master = df_ogn[df_ogn.component>=0].groupby('component').agg(master_agg_dict).reset_index()
    df_master = pd.merge(df_ogn[df_ogn.is_master],
                        df_master,
                        on='component',
                        how='inner',
                        suffixes=['','_master'])
    for c in master_agg_dict.keys():
        df_master[c] = df_master[c+'_master']
        df_master = df_master.drop(columns=c+'_master')

    return df_master

In [323]:
df_ogn = pd.concat([create_master_player_records(df_ogn), df_ogn[df_ogn.component==-1]])
df_ogn = df_ogn.drop(columns=['idx', 'component', 'idx_master', 'is_master'])
df_ogn['is_active'] = df_ogn.gamecount > 0
scd.print_dataframe_overview(df_ogn)

(9049, 24)
NaN entries:
last_login_date    350
last_game_date     350
dtype: int64


,playercode,username,email,email_verified,birth_date,age,country,country_code,first_name,last_name,...,last_game_date,gamecount,bets,wins,deposit_count,total_deposits,total_withdraws,net_loss,cleanfullname,fingerprint
0,11329782,Manasloo,kangeroo@gmx.ch,Yes,1986-09-23,36,Switzerland,CH,Dominic Christoph,Eicher,...,2022-06-19 18:16:22,31,19425.0,32725.0,37,35245,53545.0,-18300.0,christoph dominic eicher,christophdominiceicher19860923M
1,11318555,Nyuszi23,evafrehner@yahoo.co.uk,Yes,1978-03-23,44,Switzerland,CH,Eva,Frehner,...,2022-07-17 18:16:15,40,71810.0,104450.0,52,50244,86284.0,-39640.0,eva frehner,evafrehner19780323F
2,11326035,Fcsgfan1312,brian.alpiger14@gmail.com,Yes,1998-01-14,24,Switzerland,CH,Brian,Alpiger,...,2022-07-18 07:10:34,33,42889.0,41949.0,39,37245,42305.0,-8060.0,alpiger brian,alpigerbrian19980114M
3,11317661,Ste987,gamble@leimgruber.xyz,Yes,1987-08-31,35,Switzerland,CH,Stefan,Leimgruber,...,2022-07-17 18:10:40,30,55125.0,56050.0,52,50244,56169.0,-8925.0,leimgruber stefan,leimgruberstefan19870831M
4,11326893,ChrischiFr,christian.frehner@gmx.ch,Yes,1996-06-03,26,Switzerland,CH,Christian,Frehner,...,2022-07-17 18:23:21,21,28111.0,40913.0,39,37245,49045.0,-17802.0,christian frehner,christianfrehner19960603M
5,11323750,MamaEnte13,silkebarth1309@gmail.com,Yes,1973-09-13,49,Switzerland,CH,Silke,Barth,...,2022-07-17 18:32:03,13,3198.0,2668.0,25,24122,23724.0,-2470.0,barth silke,barthsilke19730913F
6,11355056,Zermatt,info@ms-schibli.ch,Yes,1962-02-18,60,Switzerland,CH,Marco,Schibli,...,2022-02-20 18:25:56,7,4700.0,8024.0,41,39245,43569.0,-4324.0,marco schibli,marcoschibli19620218M
7,11361767,Crem90,kim.desiree.seiler@gmail.com,Yes,1990-10-15,32,Switzerland,CH,Kim,Seiler,...,2022-07-17 17:36:02,20,28150.0,32100.0,29,28122,32122.0,-7950.0,kim seiler,kimseiler19901015F
8,11358909,AdoMinci,adnan.elkaz@outlook.com,Yes,1996-12-26,26,Switzerland,CH,ADNAN,ELKAZ,...,2022-05-08 18:14:47,14,35925.0,46925.0,33,31245,45245.0,-14000.0,adnan elkaz,adnanelkaz19961226M
9,11355107,gren262,beat@leuenberger.org,Yes,1957-07-24,65,Switzerland,CH,Beat,Leuenberger,...,2022-06-19 18:19:34,6,1086.0,125.0,39,37245,38284.0,-1039.0,beat leuenberger,beatleuenberger19570724M


## Matches between OGN and OC players

In [330]:
df_oc = df_oc.rename(columns={'firstname': 'first_name',
                        'lastname': 'last_name'})
df_oc['birth_date'] = df_oc.birthdate.dt.date.astype(str)
df_oc = df_oc[~df_oc.gender.isna()]

In [331]:
# preprocess to create the identity "fingerprint"
df_oc = scm.preprocess_identity_dataframe(df_oc)

# vectorize the fingerprints usinf td/idf
m_tf_idf = scm.tf_idf_vectorize(df_ogn.fingerprint.append(df_oc.fingerprint))

# these slices keep track of the original input series 
slice_ogn = slice(0, df_ogn.shape[0])
slice_oc = slice(df_ogn.shape[0], df_ogn.shape[0] + df_oc.shape[0])
# compute the best score (cosine scmilarity) matches between the dataframes  
df_best_match = scm.compute_best_cosine_matches(m_tf_idf[slice_ogn], m_tf_idf[slice_oc])

C:\Users\LEON~1.PEE\AppData\Local\Temp/ipykernel_30628/2770936205.py:5: FutureWarning:

The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



Duration:  10.46 s


In [332]:
# join the dataframes on the best match
df_match = scm.join_matches(df_best_match,
                            df_ogn,
                            df_oc.filter(['username', 'birth_date','city', 'zip', 'signupdate',
                                        'regulation_status', 'regulationstatuscode',
                                        'player_personal_sync_date', 'regulation_status_date',
                                        'cleanfullname', 'fingerprint', 'idx']))
# extend the data with two additional scores for similarity of (1) date-of-birth and (2) full-name 
df_match['dob_distance'] = df_match.apply(lambda x: scm.dob_distance(x.birth_date_m, x.birth_date_o), axis=1)
df_match['me_score'] = df_match.apply(lambda x: scm.max_monge_elkan(x.cleanfullname_m, x.cleanfullname_o), axis=1)

In [333]:
df_match['score_bin'] = pd.cut(df_match.score, np.linspace(0.5,1.05,12))
df_match['me_score_bin'] = pd.cut(df_match.me_score, np.linspace(0.5,1.05,12))
pd.pivot_table(df_match[df_match.dob_distance==0],
                index='score_bin',
                columns='me_score_bin',
                values='score',
                aggfunc='count')

me_score_bin,"(0.5, 0.55]","(0.55, 0.6]","(0.6, 0.65]","(0.65, 0.7]","(0.7, 0.75]","(0.75, 0.8]","(0.8, 0.85]","(0.85, 0.9]","(0.9, 0.95]","(0.95, 1.0]","(1.0, 1.05]"
score_bin,,,,,,,,,,,
"(0.5, 0.55]",1,6,2,1,1,2,0,0,2,10,0
"(0.55, 0.6]",0,2,2,3,0,0,1,0,4,24,0
"(0.6, 0.65]",0,1,1,1,1,1,3,3,2,45,0
"(0.65, 0.7]",0,1,0,3,5,1,1,3,4,77,0
"(0.7, 0.75]",0,0,0,0,1,3,0,4,6,145,0
"(0.75, 0.8]",0,0,0,3,1,1,0,3,12,163,0
"(0.8, 0.85]",0,0,0,0,1,2,4,10,8,185,0
"(0.85, 0.9]",0,0,0,0,0,0,0,3,10,128,0
"(0.9, 0.95]",0,0,0,0,0,0,0,0,7,41,0


In [334]:
cols_debug = ['fingerprint_m','fingerprint_o','score','me_score','cleanfullname_m','cleanfullname_o']

mask_match = ((df_match.dob_distance==0) 
            & (df_match.score.between(0.0,1.1)
            & df_match.me_score.between(0.95,1.1)))

df_match.loc[mask_match, cols_debug]

,fingerprint_m,fingerprint_o,score,me_score,cleanfullname_m,cleanfullname_o
1,evafrehner19780323F,evafrehner19780323F,1.000000,1.0,eva frehner,eva frehner
2,alpigerbrian19980114M,alpigerbrianlars19980114M,0.821365,1.0,alpiger brian,alpiger brian lars
3,leimgruberstefan19870831M,leimgruberstefan19870831M,1.000000,1.0,leimgruber stefan,leimgruber stefan
5,barthsilke19730913F,barthsilke19730913F,1.000000,1.0,barth silke,barth silke
6,marcoschibli19620218M,marcoschibli19620218M,1.000000,1.0,marco schibli,marco schibli
...,...,...,...,...,...,...
9040,alessandraklopfstein19910731F,alessandraklopfstein19910731F,1.000000,1.0,alessandra klopfstein,alessandra klopfstein
9042,notistefan19800429M,notistefan19800429M,1.000000,1.0,noti stefan,noti stefan
9043,adnandzelili19711026M,adnandzelili19711026M,1.000000,1.0,adnan dzelili,adnan dzelili
9044,coimbrapedro19860731M,coimbracortemanuelpedroreal19860731M,0.626166,1.0,coimbra pedro,coimbra corte manuel pedro real


In [335]:
df_match = df_match.filter(['playercode', 'username_m', 'email', 'birth_date_m', 'age', 'country', 'gender', 'frozen',
       'signup_date', 'last_login_date', 'last_game_date',
       'gamecount', 'bets', 'wins', 'deposit_count', 'total_deposits', 'total_withdraws', 'net_loss', 'is_active',
       'username_o', 'signupdate', 'city', 'zip',
       'regulation_status', 'regulationstatuscode', 'player_personal_sync_date', 'regulation_status_date'
       ])
df_match = df_match.rename(columns={'username_m': 'username', 'username_o': 'username_oc', 'birth_date_m': 'birth_date', 'signupdate': 'signup_date_oc'})
df_match['has_oc_match'] = mask_match
df_match.loc[~mask_match, ['username_oc', 'city', 'zip', 'signup_date_oc','regulation_status', 'regulationstatuscode', 'player_personal_sync_date', 'regulation_status_date']] = pd.NA

# Analysis

## OC matches

In [336]:
df_match.regulation_status.value_counts(dropna=False)

<NA>                       4849
verified_account           3833
closed_verified_account     320
provisional_account          47
Name: regulation_status, dtype: int64

In [340]:
df_match.groupby('has_oc_match').is_active.value_counts(dropna=False)

has_oc_match  is_active
False         True         3145
              False        1704
True          True         2504
              False        1696
Name: is_active, dtype: int64

In [345]:
df_match['ogn_to_oc_signup_day_cnt'] = (df_match.signup_date_oc.dt.date - df_match.signup_date.dt.date).dt.days
print('Number of players that were already signed up for OC:', sum(df_match.ogn_to_oc_signup_day_cnt<0))
print('Number of players that signed up for OC on the same day:', sum(df_match.ogn_to_oc_signup_day_cnt==0))
print('Number of players that signed up for OC later:', sum(df_match.ogn_to_oc_signup_day_cnt>0))

Number of players that were already signed up for OC: 3945
Number of players that signed up for OC on the same day: 166
Number of players that signed up for OC later: 89


In [346]:
fig = px.histogram(df_match, x='ogn_to_oc_signup_day_cnt', nbins=1000)
fig.show()

## Game plays

In [354]:
# map ogn identities
ogn_dates = ['2022-02-20','2022-03-20', '2022-04-18','2022-05-15', '2022-06-19', '2022-07-17']
mask_real_ogn_date = df_game.date.isin(ogn_dates)
print('Excluding non-OGN date entries:', sum(~mask_real_ogn_date))
df_game = pd.merge(df_game[mask_real_ogn_date],
                df_ogn_user_to_master,
                on='username',
                how='inner').drop(columns='username').rename(columns={'username_master': 'username'})

Excluding non-OGN date entries: 147


In [355]:
df_player_stats = df_game.groupby('username').agg({'date': ['count','min','max'],
                                                'games': ['sum','min','max'],
                                                'total_income': ['sum','min','max']}).reset_index()
df_player_stats.columns = ['username','date_cnt','min_date','max_date','tot_games_cnt','min_games_cnt','max_ganes_cnt','tot_income_chf','min_income_chf','max_income_chf']
print(df_player_stats.shape)
df_player_stats = pd.merge(df_player_stats,
    	                    df_match,
                            on='username',
                            how='inner')
print(df_player_stats.shape)

(5451, 10)
(5451, 38)


In [356]:
pd.pivot_table(df_player_stats,
                index='date_cnt',
                columns='min_date',
                values='username',
                aggfunc='count',
                fill_value='')#.to_clipboard()

min_date,2022-02-20,2022-03-20,2022-04-18,2022-05-15,2022-06-19,2022-07-17
date_cnt,,,,,,
1,689.0,913.0,526.0,333.0,294.0,522.0
2,367.0,335.0,206.0,94.0,96.0,
3,215.0,147.0,86.0,74.0,,
4,150.0,95.0,54.0,,,
5,99.0,67.0,,,,
6,89.0,,,,,


In [357]:
pd.pivot_table(df_player_stats[df_player_stats.has_oc_match],
                index='date_cnt',
                columns='min_date',
                values='username',
                aggfunc='count',
                fill_value='')#.to_clipboard()

min_date,2022-02-20,2022-03-20,2022-04-18,2022-05-15,2022-06-19,2022-07-17
date_cnt,,,,,,
1,285.0,325.0,206.0,161.0,156.0,194.0
2,184.0,137.0,99.0,53.0,42.0,
3,127.0,75.0,45.0,37.0,,
4,95.0,44.0,32.0,,,
5,57.0,40.0,,,,
6,66.0,,,,,


In [359]:
df_game.groupby('date').games.describe()

,count,mean,std,min,25%,50%,75%,max
date,,,,,,,,
2022-02-20,1609.0,7.420758,2.186023,1.0,7.0,8.0,9.0,9.0
2022-03-20,2264.0,6.542845,1.882759,1.0,6.0,7.0,8.0,15.0
2022-04-18,1819.0,6.289720,2.153657,1.0,5.0,7.0,8.0,8.0
2022-05-15,1417.0,6.602682,1.918780,1.0,6.0,7.0,8.0,8.0
2022-06-19,1143.0,6.232721,2.145352,1.0,5.0,7.0,8.0,8.0
2022-07-17,1347.0,6.195249,2.156170,1.0,5.0,7.0,8.0,8.0


In [361]:
df_game.groupby('date').total_income.describe()

,count,mean,std,min,25%,50%,75%,max
date,,,,,,,,
2022-02-20,1609.0,-1787.635177,7210.041251,-87025.0,-1814.00,200.0,2000.0,4000.0
2022-03-20,2264.0,617.618154,2853.205285,-25800.0,14.25,1200.0,2300.0,3098.5
2022-04-18,1819.0,1098.339747,2071.734956,-11400.0,232.50,1640.0,2740.0,3000.0
2022-05-15,1417.0,-1277.659845,5461.321741,-51100.0,-2200.00,200.0,2000.0,3000.0
2022-06-19,1143.0,131.970254,2436.047054,-15000.0,-655.50,645.0,1775.0,3850.0
2022-07-17,1347.0,273.556793,2166.015126,-19450.0,-600.00,680.0,1800.0,3000.0
